# Authored by Nathan Cantwell (nycantwe@umich.edu).

# **Imports, G Drive mount, and read in csv files.**
Contains:
* Imported libraries
* Datasets (csv format) and source url's.
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
import os
import time
# Comment out G Drive mount, parquet files already written.
# from google.colab import drive
# drive.mount('/content/drive')

**Importing datasets in Google Drive, sourced from:**
* Redfin (home sales)
* FRED database (Federal Reserve Economic Data, consumer price index)
* Census.gov (state and county ID codes called FIPS)

In [2]:

# https://www.redfin.com/news/data-center/
df_housing_market = pd.read_csv('https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/county_market_tracker.tsv000.gz', sep='\t')

# https://fred.stlouisfed.org/series/CPIAUCSL
df_inflation = pd.read_csv('https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23ebf3fb&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=CPIAUCSL&scale=left&cosd=1947-01-01&coed=2025-04-01&line_color=%230073e6&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2025-06-06&revision_date=2025-06-06&nd=1947-01-01')

while True:
    try:
        # Census website again having issues
        # https://www.census.gov/library/reference/code-lists/ansi.html#cou
        # https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt
        df_county_FIPS = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/national_county2020.txt', sep='|')
        break
    except Exception as e:
        time.sleep(3)

# **Dataset metrics and merging.**
Contains:
* Number of records for each data set (max 1.2M).
* Time periods of each data set (min 2012-2025).
* Reformatting census data to combined 5 digit FIPS codes.
* Reformatting county names in the housing market data to match the census data.
* Left join FIPS to housing market data on matching county names.
* Left join CPI to housing market data on matching months.
---

**Looking into each DataFrame:**
* FIPS codes are time independent.
* Home sales data is from Jan 2012 to Mar 2025.
* CPI data is Jan 1913 to Mar 2025.

In [3]:
print(f'Redfin data from {df_housing_market.PERIOD_BEGIN.min()}, to {df_housing_market.PERIOD_BEGIN.max()}, shaped: {df_housing_market.shape}')
print(f'Inflation/CPI data from {df_inflation.observation_date.min()}, to {df_inflation.observation_date.max()}, shaped: {df_inflation.shape}')
print(f'FIPS data shaped: {df_county_FIPS.shape}')

Redfin data from 2012-01-01, to 2025-04-01, shaped: (1238375, 58)
Inflation/CPI data from 1947-01-01, to 2025-04-01, shaped: (940, 2)
FIPS data shaped: (3235, 7)


**Our FIPS code data currently lacks the 5 digit representation seen in the Redfin data.**
* States are typically represented with 2 digits, for example Alabama is assigned *01*.
* Counties are typically represented with 3 digits, like *001* for Autauga County inside Alabama.
* The FIPS dataset stores these as integers, for our desired format we combine these with arithmetic and then convert the combined integer to a string and add leading zeros.
* This yields the 5 digit representation: *01001* for Autauga County Alabama.**

**To add this 5 digit FIPS code to matching rows in the Redfin data will we join on a new column that combines the county name, and the state abbreviation.**

In [4]:
print(df_county_FIPS.head(1))

# Combine state abbv and county name to match format found in Redfin housing market data.
df_county_FIPS['county_and_state'] = [
    f"{row[1][4]}, {row[1][0]}" for row in df_county_FIPS.iterrows()
    ]

# Concat state FIPS and county FIPS, add leading zeros up to 5 digits.
df_county_FIPS['county_and_state_FIPS'] = df_county_FIPS['STATEFP']*1000+df_county_FIPS['COUNTYFP']
df_county_FIPS['county_and_state_FIPS'] = df_county_FIPS['county_and_state_FIPS'].astype(str).str.zfill(5)

print(df_county_FIPS[['county_and_state','county_and_state_FIPS']].head(1))

  STATE  STATEFP  COUNTYFP  COUNTYNS      COUNTYNAME CLASSFP FUNCSTAT
0    AL        1         1    161526  Autauga County      H1        A
     county_and_state county_and_state_FIPS
0  Autauga County, AL                 01001


**Next we join the FIPS codes into the Redfin dataset:**
* A helper function is created to solve format mismatches in the title given to different counties.
* Words like "city", "county", "borough", and "municipality" were added, dropped, or interchanged when comparing Redfin and census data.

**There were 51 distinct counties not being populated on the inital left join, this mapped to 5 types of mismatch and 2 counties missing with special characters from the FIPS dataset that were added directly.**

In [5]:
# These 51 counties have inconsistent names b/w Redfin and census data sources:

# baltimore city county > baltimore city
# richmond, roanoke, fairfax, st. louis, franklin same as above.

# charlottesville > charlottesville city
# lynchburg, norfolk, portsmouth, harrisonburg, alexandria,
# virginia beach, manassas, manassas park, chesapeake,
# falls church, suffolk, hampton, fredericksburg, newport news
# staunton, winchester, salem, petersburg, waynesboro, bristol
# williamsburg hopewell, danville, colonial heights, poquoson
# martinsville, radford, buena vista, lexington, covington
# norton, galax, emporia same as above.

# juneau borough > juneau city and borough
# sitka, wrangell, yakutat same as above.

# lewis & clark county > lewis and clark county
# king & queen county same as above.

# anchorage borough > anchorage municipality
# skagway same as above.

# la salle parish > lasalle parish

# Issues with special characters, not in helper function.
# dona ana = FIPS 35013
# valdez-cordova census area = FIPS 02261

# Helper function for aligning Redfin county names to census county names.
def county_name_matching(county):
  add_city_list = [
      'charlottesville', 'lynchburg', 'norfolk', 'portsmouth', 'harrisonburg',
      'alexandria', 'virginia beach', 'manassas', 'manassas park',
      'chesapeake', 'falls church', 'suffolk', 'hampton', 'fredericksburg',
      'newport news', 'staunton', 'winchester', 'salem', 'petersburg',
      'waynesboro', 'bristol', 'williamsburg', 'hopewell', 'danville',
      'colonial heights', 'poquoson', 'martinsville', 'radford', 'buena vista',
      'lexington', 'covington', 'norton', 'galax', 'emporia'
  ]
  add_city_and_list = [
      'juneau borough', 'sitka borough', 'wrangell borough', 'yakutat borough'
  ]
  drop_county_list = [
      'baltimore city county', 'richmond city county', 'roanoke city county',
      'fairfax city county', 'st. louis city county', 'franklin city county'
  ]
  replace_borough_list = [
      'anchorage borough', 'skagway borough'
  ]

  # If the Redfin county is in the add_city_list, add 'city' to it.
  if county[:-4] in add_city_list:
    return f'{county[:-4]} city{county[-4:]}'
  # If the Redfin county is in the add_city_and_list, add 'city and' to it.
  elif county[:-4] in add_city_and_list:
    return f'{county[:-12]} city and{county[-12:]}'
  # If the Redfin county is in the drop_county_list, drop 'county' from it.
  elif county[:-4] in drop_county_list:
    return f'{county[:-11]}{county[-4:]}'
  # If the Redfin county is in replace_borough_list, change to 'municipality'.
  elif county[:-4] in replace_borough_list:
    return f'{county[:-12]} municipality{county[-4:]}'
  # Replace '&' with 'and' for 'lewis & clark', and 'king & queen' counties.
  elif '&' in county:
    return county.replace('&', 'and')
  # Remove space from 'la salle'.
  elif county=='la salle parish, la':
    return 'lasalle parish, la'
  # If not in the above cases, return original.
  else:
    return county

In [6]:
# Prepare lower case columns for matching helper function and df merge.
df_housing_market['REGION_lower'] = df_housing_market['REGION'].str.lower()
df_county_FIPS['county_and_state_lower'] = df_county_FIPS['county_and_state'].str.lower()

# Apply the county name helper function.
df_housing_market['REGION_lower'] = df_housing_market['REGION_lower'].apply(
    county_name_matching
    )

# Merge FIPS codes to housing market data, on matching county names.
df_housing_market = pd.merge(
    df_housing_market,
    df_county_FIPS[['county_and_state_lower', 'county_and_state_FIPS']],
    left_on='REGION_lower',
    right_on='county_and_state_lower',
    how='left'
    )

# Two special cases not handled by helper function, add FIPS code directly.
df_housing_market.loc[
    df_housing_market['REGION_lower']=='dona ana county, nm',
    'county_and_state_FIPS'
    ] = '35013'
df_housing_market.loc[
    df_housing_market['REGION_lower']=='valdez-cordova census area, ak',
    'county_and_state_FIPS'
    ] = '02261'

**Finally we merge CPI with the Redfin data by matching the date of CPI observation to the intial date of housing market observation data.**

In [7]:
# Convert txt values for Redfin and inflation data to datetimes
# (for period begin, end dates, inflation observation date).
df_housing_market.PERIOD_BEGIN = pd.to_datetime(df_housing_market.PERIOD_BEGIN)
df_housing_market.PERIOD_END = pd.to_datetime(df_housing_market.PERIOD_END)
df_inflation.observation_date = pd.to_datetime(df_inflation.observation_date)

# Add CPI column to housing market data.
df_housing_market = pd.merge(
    df_housing_market,
    df_inflation,
    left_on='PERIOD_BEGIN',
    right_on='observation_date',
    how='left'
    )
df_housing_market.drop('observation_date', axis=1, inplace=True)

df_housing_market[['MEDIAN_SALE_PRICE','CPIAUCSL','PERIOD_BEGIN']].head()

,MEDIAN_SALE_PRICE,CPIAUCSL,PERIOD_BEGIN
0,196000.0,301.476,2023-02-01
1,108250.0,231.015,2012-09-01
2,92000.0,260.319,2020-10-01
3,320000.0,241.176,2016-09-01
4,277700.0,307.374,2023-09-01


# **Data cleaning.**
Contains:
* Removing 52 unneeded columns.
* Sorting the df in order by: FIPS, property type, and month.
* Addressing outliers in our price columns (|z-score|>3)
* Replacing outliers with NaN's, and adding an outlier record keeping column.
* Adding monthly rows for groups (group by FIPS, by property type) without all months reported.
* Imputing NaN's by time linear interpolation, and adding an interpolation record keeping column.
* Imputing with a single fill value, forward fill, or back fill where appropriate.
* Adjusting price columns for inflation (adjusted to Jan. 2012 value).
---

**There are 52 columns unnecessary for our analysis and will be dropped. Among them are:**
* City only contains: NaN's
* Seasonal adjustment only contains: False
* Region type only contains: 'county'
* Region type ID only contains: 5
* State and state code are redundant with region
* County_and_state_lower and region_lower are redundant with region
* Period duration only contains: 30
* Table ID, and last updated (artifacts of Redfin database architecture)
* MoM and YoY columns (not part of this analysis, but can be derived later as first-order difference or ratio).


**Let's define our 9 columns of interest. Quantitative values describe the value in one county, of one property type, in a one month period:**
* FIPS code: 5 digit unique idenfitier for a US state and county
* Region: the US county and state abbreviation
* Property type: different categories of homes (single family, condo, , all, etc.)
* Period begin: start date of the month of observation
* Median sale price: central value of sale price
* Median list price: central value of list price in this county
* CPI: consumer price index, used for inflation adjustment
* Inventory: count of homes on the market
* Homes sold: count of homes sales this month/depletion of inventory.

In [8]:
df_housing_market = df_housing_market.rename(columns={
    'county_and_state_FIPS':'FIPS_CODE',
    'CPIAUCSL':'CPI'
    })

df_housing_market = df_housing_market[[
    'FIPS_CODE','REGION', 'PROPERTY_TYPE', 'PERIOD_BEGIN',
    'MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE',
    'CPI', 'INVENTORY', 'HOMES_SOLD'
    ]]

**Median sales price will be critical to this analysis. Two counties have only NaN's for this column (El Paso and Teller counties in CO).**

**While we could attempt to estimated sales price from neighboring counties, a noted ethical consideration is some municipalites by law or local custom do not disclose home sales price for privacy reasons. With respect to this possibility, these two counties are removed from the dataset.**

In [9]:
df_missing_sales = pd.merge(
    df_housing_market[df_housing_market['MEDIAN_SALE_PRICE'].isna()][['FIPS_CODE']].value_counts().to_frame(),
    df_housing_market[['FIPS_CODE']].value_counts().to_frame(),
    left_index=True,
    right_index=True,
    how='left'
    )

print(df_missing_sales.head(3))
# Dropping 2 counties without price data.
df_housing_market = df_housing_market[~df_housing_market['FIPS_CODE'].isin(['08041', '08119'])]

           0_x  0_y
FIPS_CODE          
08041      621  621
08119      396  396
20017        2  187


**Let's quickly sort our df into an intuitive order:**
1. By county FIPS
2. Then by the property type (single family, multi-family, townhouse, condo, or all)
3. Then by period start date

In [10]:
df_housing_market = df_housing_market.sort_values(by=['FIPS_CODE', 'PROPERTY_TYPE', 'PERIOD_BEGIN'])
df_housing_market = df_housing_market.reset_index().drop(columns=['index'])
df_housing_market.head(3)

,FIPS_CODE,REGION,PROPERTY_TYPE,PERIOD_BEGIN,MEDIAN_SALE_PRICE,MEDIAN_LIST_PRICE,CPI,INVENTORY,HOMES_SOLD
0,01001,"Autauga County, AL",All Residential,2012-01-01,133250.0,157450.0,227.842,286.0,20.0
1,01001,"Autauga County, AL",All Residential,2012-02-01,162000.0,185000.0,228.329,302.0,32.0
2,01001,"Autauga County, AL",All Residential,2012-03-01,149500.0,189900.0,228.807,319.0,36.0


**Looking at our price columns, they have strange min and max values, ranging from $1 to 1B. This is a good time to deal with outliers.**

In [11]:
df_housing_market.describe()

,MEDIAN_SALE_PRICE,MEDIAN_LIST_PRICE,CPI,INVENTORY,HOMES_SOLD
count,1.237338e+06,1.138006e+06,1.237358e+06,1.191991e+06,1.237358e+06
mean,2.146971e+05,2.386733e+05,2.628887e+02,2.931254e+02,8.428700e+01
std,1.081188e+06,1.346275e+06,2.852089e+01,8.651523e+02,2.696573e+02
min,1.000000e+00,1.000000e+02,2.278420e+02,1.000000e+00,1.000000e+00
25%,1.150000e+05,1.361000e+05,2.380170e+02,1.700000e+01,3.000000e+00
50%,1.697500e+05,1.924000e+05,2.542770e+02,6.300000e+01,1.300000e+01
75%,2.550000e+05,2.850000e+05,2.885820e+02,2.160000e+02,5.100000e+01
max,1.000000e+09,1.000000e+09,3.203210e+02,2.852000e+04,9.221000e+03


**For each price column we add columns for:**
* z-score
* outlier or not

**Outliers are defined with magnitude of z-score greater than 3.**

In [12]:
def add_zscore_outlier_cols(df, col):
  # Add columns to track z-score and if a value is an outlier.
  is_outlier_col = col+'_is_outlier'
  zscore_col = col+'_zscore'
  df[zscore_col] = df.groupby(['FIPS_CODE', 'PROPERTY_TYPE'])[[col]].transform(stats.zscore)
  # Define an outlier threshold here.
  max_z = 3
  df[is_outlier_col] = abs(df[zscore_col]>max_z)
  return df

In [13]:
# Add z-score and outlier columns with above helper function.
price_cols = ['MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE']
for price_col in price_cols:
  df_housing_market = add_zscore_outlier_cols(df_housing_market, price_col)

# Drop z-score columns.
price_cols_zscore = [price_col+'_zscore' for price_col in price_cols]
df_housing_market = df_housing_market.drop(columns=price_cols_zscore)

print(f'Number of median sale price outliers: {sum(df_housing_market.MEDIAN_SALE_PRICE_is_outlier)}')
print(f'Number of median list price outliers: {sum(df_housing_market.MEDIAN_LIST_PRICE_is_outlier)}')

Number of median sale price outliers: 7290
Number of median list price outliers: 1598


**It's noted that some of these outliers are likely data entry errors like a median sale price of 1B, but others may just be anomalous sales.**

This could be the case for Baldiwn County, AL in November of 2022. With 1 "homes sold" the multi-family sale was noted as 10.75M. This may be the sale of a large housing structure, but is still an unusually high outlier relative to other months of multi-family sales in Baldwin County. Both kinds of outliers are shown below. For the purposes of trend analysis we will remove both kinds of outliers.

In [14]:
df_housing_market.iloc[[699,700,701,827586,827587,827588]]

,FIPS_CODE,REGION,PROPERTY_TYPE,PERIOD_BEGIN,MEDIAN_SALE_PRICE,MEDIAN_LIST_PRICE,CPI,INVENTORY,HOMES_SOLD,MEDIAN_SALE_PRICE_is_outlier,MEDIAN_LIST_PRICE_is_outlier
699,01003,"Baldwin County, AL",Multi-Family (2-4 Unit),2022-03-01,521500.0,410000.0,287.467,11.0,10.0,False,False
700,01003,"Baldwin County, AL",Multi-Family (2-4 Unit),2022-04-01,625000.0,432671.0,288.582,6.0,2.0,False,False
701,01003,"Baldwin County, AL",Multi-Family (2-4 Unit),2022-05-01,288600.0,409000.0,291.299,6.0,4.0,False,False
827586,39137,"Putnam County, OH",Single Family Residential,2018-05-01,112000.0,115400.0,250.792,98.0,15.0,False,False
827587,39137,"Putnam County, OH",Single Family Residential,2018-06-01,114250.0,95250.0,251.018,104.0,14.0,False,False
827588,39137,"Putnam County, OH",Single Family Residential,2018-07-01,93100.0,139900.0,251.214,105.0,21.0,False,False


**We replace price column outliers with NaN's, in a new column to preserve the original data.**

In [15]:
def outliers_to_nan(df, col):
  # Add new column that removes values if they are tagged as outliers.
  is_outlier_col = col+'_is_outlier'
  no_outliers_col = col+'_no_outliers'
  df[no_outliers_col] = df[col]
  df.loc[df[is_outlier_col], no_outliers_col] = np.nan
  return df

In [16]:
# Change the price column value to NaN for outliers, with above helper function.
price_cols = ['MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE']
for price_col in price_cols:
  df_housing_market = outliers_to_nan(df_housing_market, price_col)

print(df_housing_market.isna().sum().sort_values(ascending=False).head(7))

MEDIAN_LIST_PRICE_no_outliers    100950
MEDIAN_LIST_PRICE                 99352
INVENTORY                         45367
MEDIAN_SALE_PRICE_no_outliers      7310
MEDIAN_SALE_PRICE                    20
FIPS_CODE                             0
REGION                                0
dtype: int64


**For smooth visuals, we want every county to have a row for every month. To accomplish this we merge our data to a set of columns that for each group (FIPS, and property type) has the full range of months.**

In [17]:
# Series for every unique FIPS in the dataset.
all_FIPS = pd.Series(df_housing_market['FIPS_CODE'].unique()).rename('FIPS_CODE')
# Series for every unique property type in the dataset.
all_types = pd.Series(df_housing_market['PROPERTY_TYPE'].unique()).rename('PROPERTY_TYPE')
# Series for every month between min and max values of PERIOD_BEGIN.
all_months = pd.Series(
    pd.date_range(
        start=df_housing_market['PERIOD_BEGIN'].min(),
        end=df_housing_market['PERIOD_BEGIN'].max(),
        freq='MS'# Each month, starting date.
        )
    ).rename('PERIOD_BEGIN')

# Cross join the three series above so every FIPS/property type group
# has rows for every month.
df_all_groups_all_months = pd.merge(
    all_FIPS,
    all_types,
    how='cross'
    ).merge(
        all_months,
        how='cross'
    )

# Before merging our housing market data to many new rows,
# denote current rows as original records.
df_housing_market['original_record'] = True

# Merge our housing market data onto the list of every month for every group.
df_housing_market = pd.merge(
    df_all_groups_all_months,
    df_housing_market,
    on=['FIPS_CODE', 'PROPERTY_TYPE', 'PERIOD_BEGIN'],
    how='left'
    )

**We then fill the new rows appropriately:**
* We use ***forward and back fill*** for ***county names***, on matching FIPS and property type.
* We use ***forward and back fill*** for ***CPI***, on matching months.
* We use ***forward fill*** to impute missing ***inventory*** values. It is assumed that if a county reports no sales then inventory has not changed.
* It is similary assumed with no reported sales that homes sold is zero. In the ***homes sold*** column we ***impute 0***.
* For these new rows we ***impute 'False'*** for markers of original records and outliers.

In [18]:
# For each FIPS, there is only one region value.
df_housing_market['REGION'] =\
  df_housing_market.groupby('FIPS_CODE')['REGION'].ffill()
df_housing_market['REGION'] =\
  df_housing_market.groupby('FIPS_CODE')['REGION'].bfill()
# For each month, theres is only one CPI value.
df_housing_market['CPI'] =\
  df_housing_market.groupby('PERIOD_BEGIN')['CPI'].ffill()
df_housing_market['CPI'] =\
  df_housing_market.groupby('PERIOD_BEGIN')['CPI'].bfill()
# For inventory we assume if there is no reported sales,
# then inventory is constant from last reported sales record.
df_housing_market['INVENTORY'] = \
  df_housing_market.groupby(['FIPS_CODE','PROPERTY_TYPE'])\
  ['INVENTORY'].ffill()

# Fill NaN's in "original_record" column, and "_is_outlier" columns with False.
# For homes sold column, we assume if there is no reported sales, 0 homes sold.
fill_values = {
    'original_record': False,
    'MEDIAN_SALE_PRICE_is_outlier': False,
    'MEDIAN_LIST_PRICE_is_outlier': False,
    'HOMES_SOLD': 0
    }
df_housing_market = df_housing_market.fillna(fill_values)

print(df_housing_market.isna().sum().sort_values(ascending=False).head(10))

MEDIAN_LIST_PRICE_no_outliers    1329992
MEDIAN_LIST_PRICE                1328394
MEDIAN_SALE_PRICE_no_outliers    1236352
MEDIAN_SALE_PRICE                1229062
INVENTORY                         868628
FIPS_CODE                              0
PROPERTY_TYPE                          0
PERIOD_BEGIN                           0
REGION                                 0
CPI                                    0
dtype: int64


**There are now a large number of missing values in our price and inventory columns.**

* We will be using ***linear interpolation*** to impute missing values for ***price*** in new columns to preserve the original data. Changes over time of price is a key feature of this analysis and we want to minimze NaN's. Linear imputation replaces them with a reasonable value between known values by assuming a linear relationship.
* There are some remaining missing values in inventory and price columns. These occur in the earliest months, primarily in 2012. For these, the best available technique is to ***back fill our first record for inventory, and sale and list prices***.

In [19]:
def interpolate_col(df, col):
  # Create a column to receive interpolated values.
  interpolated_col = col+'_interpolated'
  df[interpolated_col] = np.nan
  # Interpolation will be performed on the column without outliers.
  no_outliers_col = col+'_no_outliers'
  # Create a column to note these values are interpolated.
  is_interpolated_col = col+'_is_interpolated'
  df[is_interpolated_col] = df[no_outliers_col].isna()

  # From: https://stackoverflow.com/questions/37057187/pandas-interpolate-within-a-groupby
  # Define a new df to perform interpolation on.
  df_interpolated = df[['FIPS_CODE', 'PROPERTY_TYPE', 'PERIOD_BEGIN', no_outliers_col]].set_index('PERIOD_BEGIN')
  # Group by FIPS and property type, interpolate for months missing data.
  df_interpolated = df_interpolated.groupby(['FIPS_CODE', 'PROPERTY_TYPE'])\
   [['FIPS_CODE', 'PROPERTY_TYPE', no_outliers_col]].apply(
       lambda group: group.interpolate(
           method='time',
           )
       )
  # Return our original input df, with a new column of interpolated values.
  df[interpolated_col] = df_interpolated[no_outliers_col].values
  return df

In [20]:
# Add interpolated columns, with above helper function.
cols = ['MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE']
for col in cols:
  df_housing_market = interpolate_col(df_housing_market, col)

# The above fill methods and interpolation will not cover
# groups with earliest months missing (primarily in 2012),
# for these there are limited options, we will backfill.
cols = ['MEDIAN_SALE_PRICE_interpolated',
              'MEDIAN_LIST_PRICE_interpolated',
              'INVENTORY']
for col in cols:
  df_housing_market[col] = df_housing_market.groupby(
      ['FIPS_CODE','PROPERTY_TYPE']
      )[col].bfill()

print(df_housing_market.shape)
print(df_housing_market.isna().sum().sort_values(ascending=False).head(10))

(2466400, 18)
MEDIAN_LIST_PRICE_no_outliers        1329992
MEDIAN_LIST_PRICE                    1328394
MEDIAN_SALE_PRICE_no_outliers        1236352
MEDIAN_SALE_PRICE                    1229062
MEDIAN_LIST_PRICE_interpolated        695680
INVENTORY                             650720
MEDIAN_SALE_PRICE_interpolated        419360
FIPS_CODE                                  0
MEDIAN_SALE_PRICE_is_interpolated          0
original_record                            0
dtype: int64


**There are still some missing values in inventory, interpolated sale price, and interpolated list price:**
* These occur where a group has no records at all.
* For example, a county may have no reported list prices or inventory.
* Another example is counties that lack records of a particular property type entirely.
* These missing values will be left as is.

**Lastly we adjust our price columns for inflation, keeping both columns for raw price and inflation adjusted price.**

In [21]:
def inflation_adjustment(df, price_col):
  # Get our earliest record, and the CPI on that date.
  earliest_period = min(df['PERIOD_BEGIN'])
  CPI_init = df[df_housing_market['PERIOD_BEGIN']==earliest_period]['CPI'][0]

  # Create an inflation adjusted column from the price column.
  inflation_col = f'{price_col}_inflation_adj_{str(earliest_period)[0:7]}'
  inflation_adjustment_col = df['CPI']/CPI_init
  df[inflation_col] = df[price_col]*inflation_adjustment_col
  return df

In [22]:
# Add inflation adjusted columns, with above helper function.
price_cols = [
    'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_no_outliers',
    'MEDIAN_SALE_PRICE_interpolated',
    'MEDIAN_LIST_PRICE', 'MEDIAN_LIST_PRICE_no_outliers',
    'MEDIAN_LIST_PRICE_interpolated'
    ]
for price_col in price_cols:
  df_housing_market = inflation_adjustment(df_housing_market, price_col)

df_housing_market.head(3)

,FIPS_CODE,PROPERTY_TYPE,PERIOD_BEGIN,REGION,MEDIAN_SALE_PRICE,MEDIAN_LIST_PRICE,CPI,INVENTORY,HOMES_SOLD,MEDIAN_SALE_PRICE_is_outlier,...,MEDIAN_SALE_PRICE_interpolated,MEDIAN_SALE_PRICE_is_interpolated,MEDIAN_LIST_PRICE_interpolated,MEDIAN_LIST_PRICE_is_interpolated,MEDIAN_SALE_PRICE_inflation_adj_2012-01,MEDIAN_SALE_PRICE_no_outliers_inflation_adj_2012-01,MEDIAN_SALE_PRICE_interpolated_inflation_adj_2012-01,MEDIAN_LIST_PRICE_inflation_adj_2012-01,MEDIAN_LIST_PRICE_no_outliers_inflation_adj_2012-01,MEDIAN_LIST_PRICE_interpolated_inflation_adj_2012-01
0,01001,All Residential,2012-01-01,"Autauga County, AL",133250.0,157450.0,227.842,286.0,20.0,False,...,133250.0,False,157450.0,False,133250.000000,133250.000000,133250.000000,157450.000000,157450.000000,157450.000000
1,01001,All Residential,2012-02-01,"Autauga County, AL",162000.0,185000.0,228.329,302.0,32.0,False,...,162000.0,False,185000.0,False,162346.266272,162346.266272,162346.266272,185395.427533,185395.427533,185395.427533
2,01001,All Residential,2012-03-01,"Autauga County, AL",149500.0,189900.0,228.807,319.0,36.0,False,...,149500.0,False,189900.0,False,150133.190983,150133.190983,150133.190983,190704.300787,190704.300787,190704.300787


# **Reformatting and export to parquet.**
Contains:
* Rounding columns to 0 decimal places, and convert to int type.
* Reordering columns.
* Generate DF of only county aggregate values.
* Export DF's to two parquet files.
---

**Here we round off decimals for many columns. For inventory and homes sold a partial home did not make sense. For price columns we round off cents for readability of later visualizations. We then reorder columns most to least important for later analysis.**

In [23]:
# Round quantitative columns to 0 decimal places
# (no partial homes sold, whole $ values)
rounded_cols = [
    'INVENTORY', 'HOMES_SOLD',
    'MEDIAN_SALE_PRICE', 'MEDIAN_LIST_PRICE',
    'MEDIAN_SALE_PRICE_no_outliers', 'MEDIAN_LIST_PRICE_no_outliers',
    'MEDIAN_SALE_PRICE_interpolated', 'MEDIAN_LIST_PRICE_interpolated',
    'MEDIAN_SALE_PRICE_inflation_adj_2012-01',
    'MEDIAN_SALE_PRICE_no_outliers_inflation_adj_2012-01',
    'MEDIAN_SALE_PRICE_interpolated_inflation_adj_2012-01',
    'MEDIAN_LIST_PRICE_inflation_adj_2012-01',
    'MEDIAN_LIST_PRICE_no_outliers_inflation_adj_2012-01',
    'MEDIAN_LIST_PRICE_interpolated_inflation_adj_2012-01'
]

df_housing_market[rounded_cols] = df_housing_market[rounded_cols].apply(
    lambda x: round(x)
    ).astype('Int64')

# Format columns into a logical order.
df_housing_market = df_housing_market[[
    'FIPS_CODE', 'REGION',
    'PROPERTY_TYPE', 'PERIOD_BEGIN',
    'INVENTORY', 'HOMES_SOLD',
    'MEDIAN_SALE_PRICE_interpolated',
    'MEDIAN_SALE_PRICE_interpolated_inflation_adj_2012-01',
    'MEDIAN_LIST_PRICE_interpolated',
    'MEDIAN_LIST_PRICE_interpolated_inflation_adj_2012-01',
    'MEDIAN_SALE_PRICE',
    'MEDIAN_SALE_PRICE_inflation_adj_2012-01',
    'MEDIAN_LIST_PRICE',
    'MEDIAN_LIST_PRICE_inflation_adj_2012-01',
    'MEDIAN_SALE_PRICE_no_outliers',
    'MEDIAN_SALE_PRICE_no_outliers_inflation_adj_2012-01',
    'MEDIAN_LIST_PRICE_no_outliers',
    'MEDIAN_LIST_PRICE_no_outliers_inflation_adj_2012-01',
    'CPI',
    'original_record',
    'MEDIAN_SALE_PRICE_is_interpolated',
    'MEDIAN_LIST_PRICE_is_interpolated',
    'MEDIAN_SALE_PRICE_is_outlier',
    'MEDIAN_LIST_PRICE_is_outlier',
    ]]
df_housing_market.head(3)

,FIPS_CODE,REGION,PROPERTY_TYPE,PERIOD_BEGIN,INVENTORY,HOMES_SOLD,MEDIAN_SALE_PRICE_interpolated,MEDIAN_SALE_PRICE_interpolated_inflation_adj_2012-01,MEDIAN_LIST_PRICE_interpolated,MEDIAN_LIST_PRICE_interpolated_inflation_adj_2012-01,...,MEDIAN_SALE_PRICE_no_outliers,MEDIAN_SALE_PRICE_no_outliers_inflation_adj_2012-01,MEDIAN_LIST_PRICE_no_outliers,MEDIAN_LIST_PRICE_no_outliers_inflation_adj_2012-01,CPI,original_record,MEDIAN_SALE_PRICE_is_interpolated,MEDIAN_LIST_PRICE_is_interpolated,MEDIAN_SALE_PRICE_is_outlier,MEDIAN_LIST_PRICE_is_outlier
0,01001,"Autauga County, AL",All Residential,2012-01-01,286,20,133250,133250,157450,157450,...,133250,133250,157450,157450,227.842,True,False,False,False,False
1,01001,"Autauga County, AL",All Residential,2012-02-01,302,32,162000,162346,185000,185395,...,162000,162346,185000,185395,228.329,True,False,False,False,False
2,01001,"Autauga County, AL",All Residential,2012-03-01,319,36,149500,150133,189900,190704,...,149500,150133,189900,190704,228.807,True,False,False,False,False


**For affordability analysis we will look at an aggregated dataset at the county level. This is represented as the columns where property type is 'All Residential'. We separate this to its own df and csv file.**

In [24]:
# Still missing are 39 counties that do not report inventory (159 NaN's each),
# and 28 counties that do not report list price (159 NaN's each).
df_all_residential = df_housing_market[df_housing_market['PROPERTY_TYPE']=='All Residential']
df_all_residential = df_all_residential.reset_index().drop(columns='index')
print(df_all_residential.shape)
print(df_all_residential.isna().sum().sort_values(ascending=False).head(12))

(493280, 24)
MEDIAN_LIST_PRICE_no_outliers                           86392
MEDIAN_LIST_PRICE_no_outliers_inflation_adj_2012-01     86392
MEDIAN_LIST_PRICE                                       85615
MEDIAN_LIST_PRICE_inflation_adj_2012-01                 85615
MEDIAN_SALE_PRICE_no_outliers                           73170
MEDIAN_SALE_PRICE_no_outliers_inflation_adj_2012-01     73170
MEDIAN_SALE_PRICE                                       71056
MEDIAN_SALE_PRICE_inflation_adj_2012-01                 71056
INVENTORY                                                6080
MEDIAN_LIST_PRICE_interpolated                           4320
MEDIAN_LIST_PRICE_interpolated_inflation_adj_2012-01     4320
MEDIAN_SALE_PRICE_is_outlier                                0
dtype: int64


**Finally, write to parquet. Full files are over 10MB, here we create a sample of 10,000 records each. The full files were used in creating the spaghetti plots. If desired, comment out the lines for sampling to recreate the plot seen in the report.**

**(Parquet writing is commented out as the parquets are already created.)**

In [25]:
# Create sample parquet files, 10,000 records each.
df_housing_market = df_housing_market.sample(n=10000, random_state=42)
df_all_residential = df_all_residential.sample(n=10000, random_state=42)

'''
df_housing_market.to_parquet(
    '/content/drive/MyDrive/SIADS 593 Milestone I/data/final/home_price_all_types.parquet.gzip',
    compression='gzip'
    )

df_all_residential.to_parquet(
    '/content/drive/MyDrive/SIADS 593 Milestone I/data/final/home_price_agg_by_county.parquet.gzip',
    compression='gzip'
    )
'''

"\ndf_housing_market.to_parquet(\n    '/content/drive/MyDrive/SIADS 593 Milestone I/data/final/home_price_all_types.parquet.gzip',\n    compression='gzip'\n    )\n\ndf_all_residential.to_parquet(\n    '/content/drive/MyDrive/SIADS 593 Milestone I/data/final/home_price_agg_by_county.parquet.gzip',\n    compression='gzip'\n    )\n"